# Project For Detecting The Emotion On Face
its a CNN model which helps us to detect the emotions which appear on face emotions such as (happy,sad,angry,suprised,neutral) 

In [ ]:
#importing the necessery libraries 

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,BatchNormalization,Activation
from keras.layers import Conv2D,MaxPooling2D

#for opencv and final output

from keras.models import load_model
from time import sleep
from keras.preprocessing import image
import cv2
import numpy as np

In [ ]:
num_classes = 5     #represents the number of classes which are 5 (happy,sad,angry....)
rows,cols = 48,48   #rows&cols of the img 48*48
batch_size=35       

In [ ]:
train_data='/content/drive/My Drive/Colab Notebooks/train'

test_data='/content/drive/My Drive/Colab Notebooks/validation'

train_data_gen=ImageDataGenerator(rotation_range=30,
                                  shear_range=0.3,
                                  width_shift_range=0.4,
                                  height_shift_range=0.4,
                                  horizontal_flip=True,
                                  zoom_range=0.3,
                                  rescale=1./255)


test_data_gen = ImageDataGenerator(rescale=1./255)


train_gen = train_data_gen.flow_from_directory(train_data,
                                               color_mode='grayscale',
                                               target_size=(rows,cols),
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               shuffle=True)

test_gen = test_data_gen.flow_from_directory(test_data,
                                             color_mode='grayscale',
                                             target_size=(rows,cols),
                                             batch_size=batch_size,
                                             class_mode='categorical',
                                             shuffle=True)


In [ ]:
#creating model
model = Sequential()

#first_layer
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#2nd_layer
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#3rd layer
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#4th_layer
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#5th_layer
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#6th_layer
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#7th_layer

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())



In [ ]:
#importing libararies for model compilation


from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint('emotion_detector_cnn.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

reduse_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduse_lr]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

train_samples = 24266
valid_samples = 3006
epochs = 28

model_run = model.fit_generator(train_gen,
                                steps_per_epoch=train_samples//batch_size,
                                epochs=epochs,
                                callbacks=callbacks,
                                validation_data=test_gen,
                                validation_steps=valid_samples//batch_size)

In [ ]:
#final phase to show the output

face_classifer = cv2.CascadeClassifier('')
classifier = load_model('')

class_labels = ['The Angry You','The Happy you','Neutral','The Sad You','The Suprise You']
cap = cv2.VideoCapture(0)

while True:
  ret,frame=cap.read()
  labels = []
  gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  faces = face_classifer.detectMultiScale(gray,1.3,5)

  for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h,x:x+w]
    roi_gray cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

    if np.sum([roi_gray])!=0:
      roi = roi_gray.astype('float')/255.0
      roi = img_to_array(roi)
      roi = np.expand_dims(roi,axis=0)
      pred = classifier.predict(roi)[0]
      label = class_labels[pred.argmax()]
      lebel_position = (x,y)
      cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    else:
      cv2.putText(frame,'No Faces Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
  cv2.imshow('Emotion Detector',frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()


